## Localization of Dates and Times with Flask-Moment 
- pip install flask-moment

In [ ]:
from flask.ext.moment import Moment
moment = Moment(app)

from datetime import datetime

@app.route('/')
def index():
    return render_template('index.html', current_time=datetime.utcnow())

# Chapter 4 : Web Forms
- Although the support provided in Flask’s request object is sufficient for the handling of web forms, there are a number of tasks that can become tedious and repetitive. Two good examples are the generation of HTML code for forms and the validation of the submitted form data. 
- The Flask-WTF extension makes working with web forms a much more pleasant experience. This extension is a Flask integration wrapper around the framework-agnostic WTForms package. 
- pip install falsk-wtf

## Cross-Site REquest Forgery (CSRF) Protection

- To implement CSRF protection, Flask-WTF needs the application to configure an encryption key. Flask-WTF uses this key to generate encrypted tokens that are used to verify the authenticity of requests with form data. 

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'

- The app.config dictionary is a general-purpose place to store configuration variables used by the framework, the extensions, or the application itself. Configuration values can be added to the app.config object using standard dictionary syntax. The configuration object also has methods to import configuration values from files or the environment

## Form Classes

- When using Flask-WTF, each web form is represented by a class that inherits from class Form. The class defines the list of fields in the form, each represented by an object. Each field object can have one or more validators attached; validators are functions that check whether the input submitted by the user is valid. 

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import Required

class NameForm(Form):
    name = StringField("Wht is your name?'", validators=[Required()])
    submit = SubmitField('Submit')
    

![alt text](wtforms_standard_html_fields.png 'wtforms_standard_html_fields')

![alt text](wtforms_validators.png 'wtforms_validators')

## HTML Rendeing of Forms

In [ ]:
<form method="POST">
    {{ form.name.label }} {{ form.name() }}
    {{ form.submit() }}
</form>

In [ ]:
<form method="POST">
    {{ form.name.label }} {{ form.name(id='my-text-field') }}
    {{ form.submit() }}
</form>

But even with HTML attributes, the effort required to render a form in this way is significant, so it is best to leverage Bootstrap’s own set of form styles whenever possible. Flask-Bootstrap provides a very high-level helper function that renders an entire FlaskWTF form using Bootstrap’s predefined form styles, all with a single call. Using FlaskBootstrap, the previous form can be rendered as follows:


In [ ]:
{% import "bootstrap/wtf.html" as wtf %}
{{ wtf.quich_form(form)}}

The import directive works in the same way as regular Python scripts do and allows template elements to be imported and used in many templates. The imported bootstrap/ wtf.html file defines helper functions that render Flask-WTF forms using Bootstrap. 

- templates/index.html: Using Flask-WTF and Flask-Bootstrap to render a form 

<pre>
{% extends "base.html" %}
{% import "bootstrap/wtf.html" as wtf %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello, {% if name %}{{ name }}{% else %}Stranger{% endif %}!</h1>
</div>
{{ wtf.quick_form(form) }}
{% endblock %}



## Form Handling in View Functions

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    name = None
    form = NameForm()
    if form.validate_on_submit():
        name = form.name.data
        form.name.data = ''
    return render_template('index.html', form=form, name=name)


- The methods argument added to the app.route decorator tells Flask to register the view function as a handler for GET and POST requests in the URL map. When methods is not given, the view function is registered to handle GET requests only. 

- Adding POST to the method list is necessary because form submissions are much more conveniently handled as POST requests. It is possible to submit a form as a GET request, but as GET requests have no body, the data is appended to the URL as a query string and becomes visible in the browser’s address bar. For this and several other reasons, form submissions are almost universally done as POST requests. 

## Redirect and User Sessions

In [ ]:
from flask import Flask, render_template, session, redirect, url_for

@app.route('/', methods=['GET', 'POST'])
def index():
    form = NameForm()
    if form.validate_on_submit():
        session['name'] = form.name.data
        return redirect(url_for('index'))
    return render_tamplate('index.html', form=form, name=session.get('name'))

- In the previous version of the application, a local name variable was used to store the name entered by the user in the form. That variable is now placed in the user session as session['name'] so that it is remembered beyond the request. 

## Message Flashing

In [ ]:
from flask import Flask, render_template, session, redirect, url_for, flash

@app.route('/', methods=['GET', 'POST'])
def index():
    form = NameForm()
    if form.validate_on_submit():
        old_name = session.get('name')
        if old_nmae is not None old_name != form.name.data:
            flash('Looks like you have changed your name!')
        session['name'] = form.name.data
        form.name.dara = ''
        return redirect(url_for('index'))
    return render_template('index.html',
            form = form, name = sessionget('name'))

- should put get_flashed_messages() function in the base or layout html

# Chapter 5 : Databases
- SQL & NoSQL

- Easee of use 
- performance
- portability
- flask integration

## Database Management with Flask-SQLAlchemy

- pip install flask-sqlalchemy

- SQLALCHEMY_DATABASE_URI, SQLALCHEMY_COMMIT_ON_TEARDOWN

In [ ]:
from flask_sqlalchemy improt SQLAlchemy

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = \
    'sqlite:///' + os.path.join(basefir, 'data.sqlite')
app.config['SQLALCHEMY_COMMIT_ON_TEARDOWN'] = True

db = SQLAlchemy(app)


## Model Definition
- The term model is used to refer to the persistent entities used by the application. In the context of an ORM, a model is typically a Python class with attributes that match the columns of a corresponding database table. 

In [ ]:
class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    
    def __repr__(self): # for debugging and testing purpose
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    
    def __repr__(self):
        return '<User %r>' % self.username

![alt text](most_common_sqlalchemy_column_options.png 'most_common_sqlalchemy_column_options')

## Relationships
- 

In [ ]:
class Role(db.Model):
    # ...
    users = db.relationship('User', backref='role')
    
    
class User(db.Model):
    # ...
    role_id = db.Column(db.Integer, db.ForeignKey('roels.id'))

- As seen in Figure 5-1, a relationship connects two rows through the user of a foreign key. The role_id column added to the User model is defined as a foreign key, and that establishes the relationship. The 'roles.id' argument to db.ForeignKey() specifies that the column should be interpreted as having id values from rows in the roles table. 
- The users attribute added to model Role represents the object-oriented view of the relationship. Given an instance of class Role, the users attribute will return the list of users associated with that role. The first argument to db.relationship() indicates what model is on the other side of the relationship. This model can be provided as a string if the class is not yet defined. 
- The backref argument to db.relationship() defines the reverse direction of the relationship by adding a role attribute to the User model. This attribute can be used instead of role_id to access the Role model as an object instead of as a foreign key. 
- In most cases db.relationship() can locate the relationship’s foreign key on its own, but sometimes it cannot determine what column to use as a foreign key. For example, if the User model had two or more columns defined as Role foreign keys, then SQLAlchemy would not know which one of the two to use. Whenever the foreign key configuration is ambiguous, additional arguments to db.relationship() need to be given. Table 5-4 lists some of the common configuration options that can be used to define a relationship.


## Database Operations

# P. 79

- Skip, cause of too much case-focused example